In [1]:
import torch 
import os
import json
import deep_sdf
import deep_sdf.workspace as ws
import scipy.io
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.manifold import TSNE

experiment_directory = "test_experiment"
checkpoint = "210"

specs_filename = os.path.join(experiment_directory, "specs.json")

specs = json.load(open(specs_filename))

arch = __import__("networks." + specs["NetworkArch"], fromlist=["Decoder"])

latent_size = specs["CodeLength"]

decoder = arch.Decoder(latent_size, **specs["NetworkSpecs"])

decoder = torch.nn.DataParallel(decoder)
decoder.eval()

if torch.cuda.is_available():
    map_location=torch.device('cuda')
else:
    map_location=torch.device('cpu')

saved_model_state = torch.load(
    os.path.join(experiment_directory, ws.model_params_subdir, checkpoint + ".pth"
    ), map_location=map_location
)
saved_model_epoch = saved_model_state["epoch"]

decoder.load_state_dict(saved_model_state["model_state_dict"])

if torch.cuda.is_available():
    decoder = decoder.module.cuda()
else:
    decoder = decoder.module
geom_dimension = decoder.geom_dimension

In [4]:
latent_variables = []
split_filename = "./data/splits/wang_test_reduced.json"

with open(split_filename, "r") as f:
        split = json.load(f)

for dataset in split:
        for class_name in split[dataset]:
            for instance_name in split[dataset][class_name]:
                instance_filename = os.path.join(
                    "test_experiment", "Reconstructions", checkpoint,
                    "Codes", dataset, class_name, instance_name + "..pth"
                )
                print(instance_filename)
                latent = torch.load(instance_filename)
                latent_variables.append(latent)

#latent_variables = torch.cat(latent_variables, dim=0).numpy()


test_experiment/Reconstructions/210/Codes/Wang2021/class1/88393..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/49077..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/11424..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/63995..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/69421..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/48548..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/35003..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/26196..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/29693..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/84775..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/37669..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/78264..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/42674..pth
test_experiment/Reconstructions/210/Codes/Wang2021/class1/79630..pth
test_experiment/Reconstructions/21

In [ ]:

X_embedded = TSNE(n_components=2, learning_rate='auto',
                  init='random', perplexity=3).fit_transform(latent_variables)
X_embedded.shape